In [31]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [68]:
import numpy as np
import pandas as pd
import datetime as dt

#datetime has a weakness that it you need need to count days to subtract or add future dates. this module will cover that weakness
from dateutil.relativedelta import relativedelta
#using days to subtract is doable to get anything less than 1 year. But lets you want to go back 4 years from set date. You will be off if you dont factor leap year.

# Reflect Tables into SQLAlchemy ORM

In [69]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [70]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///hawaii.sqlite")

In [71]:
# reflect an existing database into a new model
Base = automap_base()
Base.prepare(engine, reflect=True)

# reflect the tables


In [72]:
# View all of the classes that automap found
Base.classes.keys()


['measurement', 'station']

In [73]:
# Save references to each table
measurement = Base.classes.measurement
station = Base.classes.station


In [74]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Precipitation Analysis

In [75]:
# Find the most recent date in the data set.

In [76]:
# I want to know the column names as written
inspector = inspect(engine)
columns = inspector.get_columns('measurement')
for c in columns:
    print(c['name'], c["type"])



id INTEGER
station TEXT
date TEXT
prcp FLOAT
tobs FLOAT


In [94]:
recent = session.query(measurement.date).order_by(measurement.date.desc()).first()

In [95]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 

# Calculate the date one year from the last date in data set.

# query_date = dt.date(2017, 8, 23) - dt.timedelta(days=365)

#both codes the same. But since I went a lot of trouble to get the second one, I am commenting out the one on top. And not delete it.

query_date = dt.date(2017, 8, 23) - relativedelta(years=1)
print(query_date)

# Perform a query to retrieve the data and precipitation scores


# Save the query results as a Pandas DataFrame and set the index to the date column


# Sort the dataframe by date


# Use Pandas Plotting with Matplotlib to plot the data




2019-08-13


In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data


# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset


In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.


In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.


In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


# Close session

In [ ]:
# Close Session
session.close()